# Redes Neurais
Usei o Kernel do Kaggle

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


### O dataset Mines vs Rocks já está no Kaggle, mas pode ser baixado daqui:  
https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar%2C+Mines+vs.+Rocks%29


In [ ]:
input_data = pd.read_csv('/kaggle/input/mines-vs-rocks/sonar.all-data.csv', header=None)
input_data.head() # uma olhada no dataset

Conta quantas minas e quantas rochas tem no dataset  #minas 0    #rochas 1

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  #codifica as letras M e R em 0 e 1
y = input_data[60].copy()
y = LabelEncoder().fit_transform(y)

X_df = input_data.copy()
X_df.drop([60], inplace=True, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_df.values, y, test_size=0.2, shuffle=True, random_state=42)

print('Training data and target sizes: \n{}, {}'.format(X_train.shape,y_train.shape))
print('Test data and target sizes: \n{}, {}'.format(X_test.shape,y_test.shape))

# Criação do Modelo

Numa aplicação real, não pode haver falso negativo, pois seria desastroso identificar uma mina como rocha. Falsos positivos são aceitáveis. Assim, coloquei peso maior (2.975) para a classe 0de forma que não houvesse falsos negativos. Valor foi obtido com tentativa e erro

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn import metrics


from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier( activation='logistic',solver='adam', 
                    max_iter = 3000, verbose=True,warm_start=False, learning_rate='constant', batch_size=145)
#Rede simples, uma única camada oculta, com 100 neurônios.
mlp.fit(X_train,y_train)

In [ ]:
y_pred = mlp.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
print("Classification report for classifier %s:\n%s\n"
     % (mlp, metrics.classification_report(y_test, y_pred)))
print("Accuracy of model = %2f%%" % (accuracy_score(y_test, y_pred )*100))
#print(classification_report(y_test, y_pred ))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_pred))

### Eliminando os falsos negativos
O modelo de redes neurais do scikit-learn não tem o hiperparâmetro class weight. Uma das maneiras de eliminar os falsos negativos é elevar o nível de decisão ao prever probabilidade. Um valor interessante é 0.95. Probabilidades abaixo desse valor serão classificadas como Minas (classe 0)

In [ ]:
y_predproba = mlp.predict_proba(X_test)[:,1]
y_predproba= y_predproba>0.95    # passa as probabilidades >0.95 para 1. O resto, 0
y_predproba=y_predproba.astype(int)

#y_predproba

print("Accuracy of model = %2f%%" % (accuracy_score(y_test, y_predproba )*100))
#print(classification_report(y_test, y_pred ))
print("Confusion matrix:\n%s" % metrics.confusion_matrix(y_test, y_predproba))

### Testando o algoritmo do zero
Testamos uma amostra com o algoritmo do zero e a mesma amostra com o algoritmo do scikit learn. Esperamos encontrar valores semelhantes

In [ ]:
coe0=np.array(mlp.coefs_[0])
coe1=np.array(mlp.coefs_[1])
inter0=np.array(mlp.intercepts_[0])
inter1=np.array(mlp.intercepts_[1])

In [ ]:
h1=1/(1+np.exp(-(np.dot(X_df.values[98].transpose(),coe0)+inter0)))   #prevendo a amostra 98
h2=1/(1+np.exp(-(np.dot(h1.transpose(),coe1)+inter1)))
h2

In [ ]:
mlp.predict_proba(X_df.values)[98]

### Download dos coeficientes
Algoritmo verificado, basta implementarmos em C no Arduíno, utilizando os coeficientes. Para facilitar, usaremos o código abaixo e exportamos para um arquivo texto. No Kaggle, é só fazer o commit e buscar os dados em Output.

In [ ]:
np.savetxt('NNCOE0.txt',coe0, delimiter=',', fmt='%f') #Salvar os coeficientes da rede neural
np.savetxt('NNCOE1.txt',coe1, delimiter=',', fmt='%f') 
np.savetxt('NNINTERCEPTS0.txt',inter0, delimiter=',', fmt='%f') 
np.savetxt('NNINTERCEPTS1.txt',inter1, delimiter=',', fmt='%f') 